In [17]:
# YOLO 데이터셋 통계 (ipynb 셀에서 실행)
# 대상 폴더: /home/dw/ws_job_msislab/Golf_Project/data/old_data/Data_20250721_fix_label

from pathlib import Path
from collections import Counter
import pandas as pd
from IPython.display import display, Markdown

# === 제목 출력 ===
display(Markdown("## 📌 21th data YOLO Dataset Statistics"))

# === 경로 설정 ===
BASE = Path("/home/dw/ws_job_msislab/Golf_Project/data/good_data_2_aug")

# === 이미지 확장자 ===
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".JPG", ".JPEG", ".PNG", ".BMP"}

# === 이미지 파일 수집 (재귀) ===
all_images = [p for p in BASE.rglob("*") if p.suffix in IMG_EXTS]

def guess_label_path(img_path: Path) -> Path:
    return img_path.with_suffix(".txt")

def parse_yolo_labels(label_path: Path):
    cls_ids = []
    try:
        with open(label_path, "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5 and parts[0].isdigit():
                    cls_ids.append(int(parts[0]))
    except FileNotFoundError:
        pass
    return cls_ids

# === 카운팅 ===
per_class_objects = Counter()
images_with_objects = 0
images_background = 0
found_class_ids = set()

for img in all_images:
    lbl = guess_label_path(img)
    cls_ids = parse_yolo_labels(lbl)
    if len(cls_ids) == 0:
        images_background += 1
    else:
        images_with_objects += 1
        per_class_objects.update(cls_ids)
        found_class_ids.update(cls_ids)

total_images = len(all_images)

# === 클래스 이름 매핑 ===
class_name_map = {
    0: "Divot",
    1: "Fixed_Divot",
    2: "Diseased grass",
}

# === 결과 DataFrame ===
overall_df = pd.DataFrame([
    {"Metric": "Total images", "Value": total_images},
    {"Metric": "Images with at least 1 object", "Value": images_with_objects},
    {"Metric": "Background images (no labels)", "Value": images_background},
])

class_ids_to_show = sorted(set(per_class_objects.keys()) | set(class_name_map.keys()))
per_class_df = pd.DataFrame(
    [{
        "Class ID": cid,
        "Class Name": class_name_map.get(cid, "(unknown)"),
        "Object Count": int(per_class_objects.get(cid, 0)),
    } for cid in class_ids_to_show]
).sort_values("Class ID").reset_index(drop=True)

# === 표 출력 ===
display(overall_df.style.set_caption("📊 YOLO – Overall Counts")
        .set_table_styles([{'selector':'th', 'props':[('text-align','center')]}])
        .hide(axis='index'))

display(per_class_df.style.set_caption("📊 YOLO – Per-class Object Counts")
        .set_table_styles([{'selector':'th', 'props':[('text-align','center')]}])
        .hide(axis='index'))

# === 추가 경고 ===
if found_class_ids - set(class_name_map.keys()):
    print("⚠️ 정의(0/1/2) 밖의 클래스 ID가 발견됨:", sorted(found_class_ids - set(class_name_map.keys())))



## 📌 21th data YOLO Dataset Statistics

Metric,Value
Total images,2863
Images with at least 1 object,2555
Background images (no labels),308


Class ID,Class Name,Object Count
0,Divot,3500
1,Fixed_Divot,3500
2,Diseased grass,300


In [11]:
from pathlib import Path
import cv2
import pandas as pd

# === 데이터 경로 ===
BASE = Path("/home/dw/ws_job_msislab/Golf_Project/data/old_data/Data_20250721_fix_label")

# === 이미지 확장자 ===
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".JPG", ".JPEG", ".PNG", ".BMP"}

# === 이미지 파일 목록 ===
all_images = [p for p in BASE.rglob("*") if p.suffix in IMG_EXTS]

bad_images = []
for img_path in all_images:
    try:
        img = cv2.imread(str(img_path))
        if img is None:
            bad_images.append(img_path)
    except Exception:
        bad_images.append(img_path)

# === 표(DataFrame)로 요약 ===
summary_df = pd.DataFrame({
    "Total Images": [len(all_images)],
    "Bad Images": [len(bad_images)],
    "Valid Images": [len(all_images) - len(bad_images)]
})

print(summary_df.to_string(index=False))


 Total Images  Bad Images  Valid Images
         5834        1759          4075
